In [71]:
# Import the dependencies.
import pandas as pd
import numpy as np
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# looks for nearest city with geo coordinates
from citipy import citipy

In [72]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [73]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [74]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

754

In [75]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=3ef41da726500c6f691a33dccaeb7ff7


In [76]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [77]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        #city_weather_condition = for item in city_weather["weather"]:print(item["description"])
        for item in city_weather["weather"]:
            city_weather_condition = item["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_condition})
       
    
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | port lincoln
Processing Record 2 of Set 1 | port alfred
Processing Record 3 of Set 1 | college
Processing Record 4 of Set 1 | saint-augustin
Processing Record 5 of Set 1 | marawi
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | nguiu
City not found. Skipping...
Processing Record 8 of Set 1 | busselton
Processing Record 9 of Set 1 | maniitsoq
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | ketchikan
Processing Record 13 of Set 1 | bethel
Processing Record 14 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 15 of Set 1 | vestmannaeyjar
Processing Record 16 of Set 1 | pathein
Processing Record 17 of Set 1 | nanortalik
Processing Record 18 of Set 1 | meulaboh
Processing Record 19 of Set 1 | hobart
Processing Record 20 of Set 1 | yellowknife
Processing Record 21 of Set 1 | fort stewart
Processing Record 22 of Set 1 | lasa
Processing Record 23 of Set 1 | blu

In [78]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Lincoln,AU,-34.7333,135.8667,54.75,74,100,15.66,overcast clouds
1,Port Alfred,ZA,-33.5906,26.8910,63.18,73,0,5.01,clear sky
2,College,US,64.8569,-147.8028,55.22,79,75,0.00,broken clouds
3,Saint-Augustin,CA,51.2260,-58.6502,72.59,52,8,14.92,clear sky
4,Marawi,PH,7.9986,124.2928,69.89,95,100,1.83,overcast clouds
...,...,...,...,...,...,...,...,...,...
685,Pangody,RU,65.8500,74.4859,47.75,74,100,12.48,overcast clouds
686,Izumo,JP,35.3667,132.7667,80.55,91,89,18.54,overcast clouds
687,Janakkala,FI,60.9000,24.6000,83.80,35,59,6.31,broken clouds
688,Molteno,ZA,-31.3968,26.3625,58.48,31,0,11.79,clear sky


In [79]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")